In [23]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from WickContractions.wick.contract import *
from src.laph_diagram import *

import os

from src.utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))

## Operators

First we have to define the operators

In [30]:
from src.op_elementals import *

aOps=[Operator([MesonBaryonSource(1,{'flavor': ['u','u','u'], 'x': 'x_0', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['u','d'], 'x':'x_1', 'gamma': '\\Gamma^M'})])]

cOps=[Operator([MesonBaryonSink(1,{'flavor': ['u','u','u'], 'x': 'x_2', 'gamma': '\\Gamma^B'},
                                    {'flavor': ['d','u'], 'x':'x_3', 'gamma': '\\Gamma^M'})])]

In [31]:
pprint(aOps[0])
pprint(cOps[0])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [32]:
diagrams=[]
for i,aOp in enumerate(aOps):
    for j,cOp in enumerate(cOps):
        for d in contract(aOp, cOp).diagrams:
            diagrams.append(d)

print("Need to compute {} diagrams".format(len(diagrams)))
pprint("\\text{{diagram}}[0]={}".format(diagrams[0]))

Need to compute 24 diagrams


<IPython.core.display.Math object>

In [34]:
laphDiagrams = [LDiagram(d) for d in copy.deepcopy(diagrams)]

for d in laphDiagrams:
    d.create_T_blocks()

pprint(laphDiagrams[0])

<IPython.core.display.Math object>